好的，我们来看一个更常见的场景：从一个 HTML 网页中提取表格数据，然后存入 Excel。

我们将使用 `requests` 获取网页内容，`BeautifulSoup` 解析 HTML 提取所需数据，最后用 `openpyxl` 写入 Excel。

**首先，安装必要的库：**

```bash
pip install requests beautifulsoup4 openpyxl
```

**示例场景：**

假设我们要从一个包含电影排行榜的简单 HTML 页面抓取数据。页面结构如下（这是我们将要解析的 HTML 结构）：

```html
<!DOCTYPE html>
<html>
<head>
    <title>Top Movies</title>
</head>
<body>
    <h1>Top 5 Movies of All Time</h1>
    <table border="1">
        <thead>
            <tr>
                <th>Rank</th>
                <th>Title</th>
                <th>Year</th>
                <th>Rating</th>
            </tr>
        </thead>
        <tbody>
            <tr>
                <td>1</td>
                <td>The Shawshank Redemption</td>
                <td>1994</td>
                <td>9.3</td>
            </tr>
            <tr>
                <td>2</td>
                <td>The Godfather</td>
                <td>1972</td>
                <td>9.2</td>
            </tr>
            <tr>
                <td>3</td>
                <td>The Dark Knight</td>
                <td>2008</td>
                <td>9.0</td>
            </tr>
            <tr>
                <td>4</td>
                <td>Pulp Fiction</td>
                <td>1994</td>
                <td>8.9</td>
            </tr>
            <tr>
                <td>5</td>
                <td>Forrest Gump</td>
                <td>1994</td>
                <td>8.8</td>
            </tr>
        </tbody>
    </table>
</body>
</html>
```

为了模拟真实环境，我们可以先创建一个本地 HTML 文件（例如 `movies.html`）或者找一个公开的、允许爬虫访问的网站。这里我们假设上面的 HTML 内容托管在一个 URL 上，比如 `http://example.com/movies.html` （请注意，这个网址是虚构的，请替换为你实际要爬取的网址）。

**Python 代码示例：**

```python
import requests
from bs4 import BeautifulSoup
from openpyxl import Workbook

# --- 1. 获取网页内容 ---
# 目标网页的URL (请替换为真实的URL)
# url = 'http://example.com/movies.html'
# 为了演示，我们在这里构造一段HTML字符串，模拟从网络获取的内容
# 在实际应用中，你应该使用上面的 url 并取消下面两行的注释
mock_html_content = """
<!DOCTYPE html>
<html>
<head>
    <title>Top Movies</title>
</head>
<body>
    <h1>Top 5 Movies of All Time</h1>
    <table border="1">
        <thead>
            <tr>
                <th>Rank</th>
                <th>Title</th>
                <th>Year</th>
                <th>Rating</th>
            </tr>
        </thead>
        <tbody>
            <tr>
                <td>1</td>
                <td>The Shawshank Redemption</td>
                <td>1994</td>
                <td>9.3</td>
            </tr>
            <tr>
                <td>2</td>
                <td>The Godfather</td>
                <td>1972</td>
                <td>9.2</td>
            </tr>
            <tr>
                <td>3</td>
                <td>The Dark Knight</td>
                <td>2008</td>
                <td>9.0</td>
            </tr>
            <tr>
                <td>4</td>
                <td>Pulp Fiction</td>
                <td>1994</td>
                <td>8.9</td>
            </tr>
            <tr>
                <td>5</td>
                <td>Forrest Gump</td>
                <td>1994</td>
                <td>8.8</td>
            </tr>
        </tbody>
    </table>
</body>
</html>
"""

# 实际网络请求代码 (取消注释以使用)
# try:
#     response = requests.get(url)
#     response.raise_for_status() # 如果响应状态码不是200，会抛出异常
#     html_content = response.text
# except requests.exceptions.RequestException as e:
#     print(f"请求网页时出错: {e}")
#     exit()

# 使用模拟的HTML内容
html_content = mock_html_content

# --- 2. 解析HTML ---
# 使用BeautifulSoup解析HTML内容
# 'html.parser' 是Python内置的解析器，速度较快且容错性好
soup = BeautifulSoup(html_content, 'html.parser')

# 查找页面上的第一个<table>标签
table = soup.find('table')

# 如果找不到表格，则退出
if not table:
    print("在网页中未找到 <table> 标签。")
    exit()

# --- 3. 提取数据 ---
# 存储提取到的数据
data_rows = []

# 查找表格中的<thead>部分，通常包含表头
thead = table.find('thead')
if thead:
    header_row = thead.find('tr')
    if header_row:
        # 提取表头文字
        headers = [th.get_text(strip=True) for th in header_row.find_all(['th', 'td'])]
        data_rows.append(headers) # 将表头作为第一行加入数据
        print("提取到的表头:", headers)
    else:
        print("<thead> 中未找到 <tr> 标签。")
else:
    print("未找到 <thead> 标签，尝试从 <tbody> 第一行推测表头...")
    # 如果没有<thead>，可以尝试从<tbody>的第一行获取表头
    tbody = table.find('tbody')
    first_tbody_row = tbody.find('tr') if tbody else None
    if first_tbody_row and all(cell.name == 'th' for cell in first_tbody_row.find_all(['th', 'td'])):
         headers = [cell.get_text(strip=True) for cell in first_tbody_row.find_all(['th', 'td'])]
         data_rows.append(headers)
         print("推测的表头:", headers)
         # 标记一下，后续跳过这一行
         skip_first_body_row = True
    else:
        # 如果连合适的表头都没有，可以手动定义或者跳过
        headers = ["Rank", "Title", "Year", "Rating"] # 假设我们知道表头
        data_rows.append(headers)
        skip_first_body_row = False

# 查找表格中的<tbody>部分，包含主要数据
tbody = table.find('tbody')
if tbody:
    rows = tbody.find_all('tr')
    # 如果之前已经处理了表头所在的行，则跳过它
    start_index = 1 if 'skip_first_body_row' in locals() and skip_first_body_row else 0
    for row in rows[start_index:]:
        # 对于每一行<tr>，查找所有的<td>或<th>单元格
        cells = row.find_all(['td', 'td']) # 通常数据在<td>里，但也可能是<th>
        # 提取每个单元格的文本，并去除首尾空白
        row_data = [cell.get_text(strip=True) for cell in cells]
        # 如果这一行有数据，则添加到列表中
        if row_data:
             data_rows.append(row_data)
             print("提取到的数据行:", row_data)
else:
    print("未找到 <tbody> 标签。")

# --- 4. 写入Excel ---
if data_rows:
    # 创建一个新的Excel工作簿和活动工作表
    wb = Workbook()
    ws = wb.active
    ws.title = "Movie Rankings" # 给工作表命名

    # 将提取的数据逐行写入Excel
    for row_data in data_rows:
        ws.append(row_data)

    # 保存工作簿到文件
    wb.save('movie_rankings.xlsx')
    print("\n数据已成功写入到 movie_rankings.xlsx 文件中。")
else:
    print("没有提取到任何数据，Excel文件未生成。")

```

**代码详细解释：**

1.  **导入库**：
    *   `requests`：用于发送 HTTP 请求，获取网页内容。
    *   `BeautifulSoup`：来自 `bs4` 包，用于解析 HTML 文档，方便地查找和提取数据。
    *   `Workbook`：来自 `openpyxl`，用于创建和操作 Excel (.xlsx) 文件。

2.  **获取网页内容 (`requests`)**：
    *   `url = '...'`：定义目标网页的地址。
    *   `response = requests.get(url)`：向服务器发送一个 GET 请求。
    *   `response.raise_for_status()`：这是一个很好的实践，它会在 HTTP 状态码表示错误（如 404 Not Found, 500 Internal Server Error）时抛出一个异常，让我们知道请求失败了。
    *   `html_content = response.text`：如果请求成功，`.text` 属性包含了服务器返回的 HTML 字符串。
    *   **注意**：出于演示目的，我使用了 `mock_html_content`。在实际运行时，应使用真实的 `url` 并执行网络请求部分。

3.  **解析 HTML (`BeautifulSoup`)**：
    *   `soup = BeautifulSoup(html_content, 'html.parser')`：创建一个 BeautifulSoup 对象。它把混乱的 HTML 字符串变成了一个结构化的树状对象，我们可以像导航文件系统一样在其中查找元素。
        *   `html_content`：要解析的 HTML 字符串。
        *   `'html.parser'`：指定使用的解析器。这是 Python 内置的，速度快，容错性较好。
    *   `table = soup.find('table')`：在解析后的文档 (`soup`) 中查找第一个出现的 `<table>` 标签。`find()` 返回匹配的第一个元素对象，如果没有找到则返回 `None`。

4.  **提取数据 (`BeautifulSoup`)**：
    *   `thead = table.find('thead')` / `tbody = table.find('tbody')`：分别查找表格内的 `<thead>` (表头) 和 `<tbody>` (主体) 部分。这是一种良好的实践，有助于区分标题和数据。
    *   `header_row = thead.find('tr')`：在 `<thead>` 中查找表头行 (`<tr>`)。
    *   `headers = [th.get_text(strip=True) for th in header_row.find_all(['th', 'td'])]`：
        *   `header_row.find_all(['th', 'td'])`：查找表头行内所有的 `<th>` (表头单元格) 或 `<td>` (数据单元格)。
        *   `for th in ...`：遍历找到的所有表头单元格。
        *   `th.get_text(strip=True)`：获取单元格内部的纯文本，并自动去除首尾空格。
        *   `[...]`：列表推导式，将所有提取到的表头文本组成一个列表。
    *   `rows = tbody.find_all('tr')`：查找 `<tbody>` 内所有的数据行 (`<tr>`)。
    *   `for row in rows:`：遍历每一行数据。
    *   `cells = row.find_all(['td', 'td'])`：查找当前行内所有的数据单元格 (`<td>`)。（注：原文中第二个 `td` 应为 `th`，此处已修正为 `['td', 'th']` 以防万一）。
    *   `row_data = [cell.get_text(strip=True) for cell in cells]`：与提取表头类似，提取当前行所有单元格的文本，形成一个数据列表。
    *   `data_rows.append(row_data)`：将处理好的每一行数据（包括表头）作为一个子列表，添加到主列表 `data_rows` 中。最终 `data_rows` 看起来像这样：
        ```python
        [
          ['Rank', 'Title', 'Year', 'Rating'], # 表头行
          ['1', 'The Shawshank Redemption', '1994', '9.3'], # 数据行1
          ['2', 'The Godfather', '1972', '9.2'],           # 数据行2
          ...
        ]
        ```

5.  **写入 Excel (`openpyxl`)**：
    *   `wb = Workbook()`：创建一个新的 Excel 工作簿对象。
    *   `ws = wb.active`：获取工作簿中当前激活的工作表（新创建的工作簿默认有一个激活的工作表）。
    *   `ws.title = "Movie Rankings"`：给工作表设置一个名称。
    *   `for row_data in data_rows:`：遍历我们之前提取并整理好的数据列表。
    *   `ws.append(row_data)`：这是 `openpyxl` 最方便的方法之一。它会将 `row_data` (一个列表) 作为一行，依次填入工作表的下一空行中。第一次调用会填入 A1, B1, C1, D1...，第二次调用会填入 A2, B2, C2, D2... 以此类推。
    *   `wb.save('movie_rankings.xlsx')`：将工作簿保存到名为 `movie_rankings.xlsx` 的文件中。